## 📈 Technical Analysis of Apple (AAPL) Stock

This Jupyter Notebook performs a comprehensive technical analysis on Apple's (AAPL) stock price data over the past five years. We will gather historical data, calculate key technical indicators, and visualize the findings to identify potential trading signals and trends.

### 💻 Setup and Data Collection

First, we'll install and import the necessary libraries. The **`yfinance`** library is excellent for downloading data directly from Yahoo Finance. We'll define our stock ticker and a five-year date range.

In [ ]:
# Install required libraries if you haven't already
# !pip install yfinance pandas numpy matplotlib mplfinance

# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf

# Set a style for the plots for better visualization
plt.style.use('fivethirtyeight')

# Define the stock ticker and date range
ticker = 'AAPL'
start_date = '2020-08-27'
end_date = '2025-08-27'

# Fetch historical data
try:
    data = yf.download(ticker, start=start_date, end=end_date)
    print(f"Data for {ticker} successfully downloaded.")
    print(data.head())
except Exception as e:
    print(f"Error fetching data: {e}")

### 🧹 Data Cleaning and Preparation

We'll check for missing values and confirm the date column is set as the index in the correct format, which `yfinance` handles automatically.

In [ ]:
# Check for missing values
print("\nChecking for missing values:")
print(data.isnull().sum())

# Confirm the 'Date' column is the index and in a datetime format
print(f"\nIndex type: {data.index.dtype}")
print(f"Column dtypes: \n{data.dtypes}")

***

### 📊 Technical Indicators Calculation

Here, we will calculate several key indicators to help us analyze the stock's momentum and potential trading signals.

#### Simple Moving Average (SMA) & Exponential Moving Average (EMA)

Moving averages help smooth out price data to identify the trend direction. We will use short-term (20-day) and long-term (50-day) periods.

In [ ]:
# Calculate SMAs and EMAs
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()
data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()

# Drop initial rows with NaN values resulting from the moving average calculations
data.dropna(inplace=True)
print("\nData with moving averages added:")
print(data.tail())

#### Relative Strength Index (RSI)

The **RSI** is a momentum oscillator from 0 to 100, used to identify **overbought** (RSI > 70) or **oversold** (RSI < 30) conditions. We will use a standard 14-day window.

In [ ]:
def calculate_rsi(data, window=14):
    """Calculates the Relative Strength Index (RSI)"""
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['RSI'] = calculate_rsi(data, window=14)
print("\nData with RSI added:")
print(data.tail())

#### Moving Average Convergence Divergence (MACD)

The **MACD** is a trend-following momentum indicator. It's the difference between the 12-period and 26-period EMA. A 9-period EMA of the MACD itself forms the **signal line**.

In [ ]:
# Calculate MACD
exp1 = data['Close'].ewm(span=12, adjust=False).mean()
exp2 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = exp1 - exp2
data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

print("\nData with MACD and Signal Line added:")
print(data.tail())

***

### 📈 Data Visualization

Visualizing the data is crucial for technical analysis. We'll use several plots to illustrate the stock's price movements and the calculated indicators.

#### Stock Price and Moving Averages

This plot shows the raw stock price alongside the SMA and EMA lines, which can act as **dynamic support and resistance levels**.

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(data['Close'], label='Close Price', color='black')
plt.plot(data['SMA_20'], label='20-Day SMA', color='orange', linestyle='--')
plt.plot(data['SMA_50'], label='50-Day SMA', color='purple', linestyle='--')
plt.plot(data['EMA_20'], label='20-Day EMA', color='cyan')
plt.plot(data['EMA_50'], label='50-Day EMA', color='magenta')
plt.title(f'{ticker} Stock Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

#### RSI and MACD Subplots

This setup provides a comprehensive view by placing the RSI and MACD plots directly below the main price chart.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16, 12), sharex=True)

# Plot 1: Close Price with Moving Averages
ax1.plot(data['Close'], label='Close Price', color='black')
ax1.plot(data['SMA_20'], label='20-Day SMA', color='orange', linestyle='--')
ax1.plot(data['SMA_50'], label='50-Day SMA', color='purple', linestyle='--')
ax1.set_title(f'{ticker} Stock Price and Moving Averages')
ax1.set_ylabel('Price (USD)')
ax1.legend(loc='upper left')
ax1.grid(True)

# Plot 2: RSI
ax2.plot(data['RSI'], label='RSI', color='blue')
ax2.axhline(70, linestyle='--', color='red', label='Overbought (70)')
ax2.axhline(30, linestyle='--', color='green', label='Oversold (30)')
ax2.set_title('Relative Strength Index (RSI)')
ax2.set_ylabel('RSI')
ax2.legend(loc='upper left')
ax2.grid(True)

# Plot 3: MACD
ax3.plot(data['MACD'], label='MACD Line', color='blue')
ax3.plot(data['Signal_Line'], label='Signal Line', color='red', linestyle='--')
ax3.set_title('MACD')
ax3.set_xlabel('Date')
ax3.set_ylabel('Value')
ax3.legend(loc='upper left')
ax3.grid(True)
plt.tight_layout()
plt.show()

#### Candlestick Chart

A **candlestick chart** provides a more detailed look at daily price movements, showing open, high, low, and close prices. This is excellent for identifying specific **chart patterns**.

In [ ]:
# Create a candlestick chart
mpf.plot(data, type='candle', style='yahoo', 
         title=f'{ticker} Candlestick Chart', 
         ylabel='Price', 
         volume=True, 
         figratio=(15, 8))

***

### 📝 Analysis and Interpretation

#### Trends and Patterns

The charts show that AAPL has been in a strong **long-term uptrend** over the past five years. Its price generally stays above the long-term moving averages, and pullbacks often find support at these lines. This indicates a consistent bullish sentiment.

#### Trading Signals from Indicators

- **Moving Average Crossovers:** A **golden cross** (20-day SMA crossing above 50-day SMA) is a **bullish signal** and a potential buy indicator. The opposite, a **death cross**, is a **bearish signal**.
- **RSI:** When the **RSI is above 70**, it suggests the stock is overbought and may be due for a correction. When the **RSI falls below 30**, it suggests the stock is oversold and could be a buying opportunity.
- **MACD:** A **bullish crossover** occurs when the MACD line crosses above the signal line, indicating increasing upward momentum. A **bearish crossover** suggests downward momentum is gaining.

#### Summary for Investors

Based on this technical analysis, Apple has demonstrated a robust, long-term upward trend. Investors can use the moving averages as dynamic support levels for identifying good entry points. The RSI and MACD indicators provide additional confirmation for short-term trading signals. However, remember that these tools are best used in conjunction with fundamental analysis and a thorough understanding of market conditions. This analysis is for educational purposes only and should not be considered financial advice.

***

### 📚 References

* **Yahoo Finance:** Data source for historical stock prices.
* **Pandas & NumPy:** Essential libraries for data manipulation and numerical operations.
* **Matplotlib & mplfinance:** Libraries used for data visualization.
* **Technical Analysis Concepts:** The methodology used is based on standard technical analysis principles as described in works such as *Technical Analysis of the Financial Markets* by John J. Murphy.